In [1]:
import json
import base64
import openai
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
from uuid import uuid4
from random import randint
from utils.get_openai_api_key import get_test_key
from utils.get_postgres_connection import _conn_open
from utils.load_json import load_json
from rich.console import Console
import psycopg2

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mrcra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
print("Connecting to PostgreSQL...")
conn = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
)

cur = conn.cursor()

print("Successfully connected to PostgreSQL.")

Connecting to PostgreSQL...
Successfully connected to PostgreSQL.


In [3]:
id = "051a2523-b8c4-4958-b166-c2fa8fb89059"
id = "1b22edff-bec9-40e8-8c8b-515655dd0377"

In [4]:
print("Getting number of characters in text field...")
sql = f"""
SELECT element_text
FROM tbl_doc_elements
WHERE element_type IN ('NARRATIVETEXT', 'TITLE', 'SUBTITLE','HEADER', 'FOOTER') 
AND id = '{id}';
"""
print(sql)

Getting number of characters in text field...

SELECT element_text
FROM tbl_doc_elements
WHERE element_type IN ('NARRATIVETEXT', 'TITLE', 'SUBTITLE','HEADER', 'FOOTER') 
AND id = '1b22edff-bec9-40e8-8c8b-515655dd0377';



In [5]:
cur.execute(sql)
text = cur.fetchone()[0]

num_chars = len(text)
print(text)
print(f"Num in text field {id[:8]}... is {num_chars:,}")

Further research is necessary to explore the safety, security and societal impacts of AI agents and multi-agent systems, emphasizing both technical solutions and organizational governance frameworks. These efforts are critical for mitigating risks associated with the ongoing development, deployment and increasing use of more sophisticated AI agents in a range of domains.
Num in text field 1b22edff... is 373


In [6]:
text = """In intelligence and artificial intelligence, an intelligent agent (IA) is an agent that perceives its environment, takes actions autonomously in order to achieve goals, and may improve its performance with learning or acquiring knowledge.

An intelligent agent may be simple or complex: A thermostat or other control system is considered an example of an intelligent agent, as is a human being, as is any system that meets the definition, such as a firm, a state, or a biome.[1]


Simple reflex agent diagram
Leading AI textbooks define "artificial intelligence" as the "study and design of intelligent agents", a definition that considers goal-directed behavior to be the essence of intelligence. Goal-directed agents are also described using a term borrowed from economics, "rational agent".[1]

An agent has an "objective function" that encapsulates all the IA's goals. Such an agent is designed to create and execute whatever plan will, upon completion, maximize the expected value of the objective function.[2]

For example, a reinforcement learning agent has a "reward function" that allows the programmers to shape the IA's desired behavior,[3] and an evolutionary algorithm's behavior is shaped by a "fitness function".[4]

Intelligent agents in artificial intelligence are closely related to agents in economics, and versions of the intelligent agent paradigm are studied in cognitive science, ethics, and the philosophy of practical reason, as well as in many interdisciplinary socio-cognitive modeling and computer social simulations.
In intelligence and artificial intelligence, an intelligent agent (IA) is an agent that perceives its environment, takes actions autonomously in order to achieve goals, and may improve its performance with learning or acquiring knowledge.

An intelligent agent may be simple or complex: A thermostat or other control system is considered an example of an intelligent agent, as is a human being, as is any system that meets the definition, such as a firm, a state, or a biome.[1]


Simple reflex agent diagram
Leading AI textbooks define "artificial intelligence" as the "study and design of intelligent agents", a definition that considers goal-directed behavior to be the essence of intelligence. Goal-directed agents are also described using a term borrowed from economics, "rational agent".[1]

An agent has an "objective function" that encapsulates all the IA's goals. Such an agent is designed to create and execute whatever plan will, upon completion, maximize the expected value of the objective function.[2]

For example, a reinforcement learning agent has a "reward function" that allows the programmers to shape the IA's desired behavior,[3] and an evolutionary algorithm's behavior is shaped by a "fitness function".[4]

Intelligent agents in artificial intelligence are closely related to agents in economics, and versions of the intelligent agent paradigm are studied in cognitive science, ethics, and the philosophy of practical reason, as well as in many interdisciplinary socio-cognitive modeling and computer social simulations.
Philosophically, this definition of artificial intelligence avoids several lines of criticism. Unlike the Turing test, it does not refer to human intelligence in any way. Thus, there is no need to discuss if it is "real" vs "simulated" intelligence (i.e., "synthetic" vs "artificial" intelligence) and does not indicate that such a machine has a mind, consciousness or true understanding. It seems not imply John Searle's "strong AI hypothesis". It also doesn't attempt to draw a sharp dividing line between behaviors that are "intelligent" and behaviors that are "unintelligent"—programs need only be measured in terms of their objective function.

More importantly, it has a number of practical advantages that have helped move AI research forward. It provides a reliable and scientific way to test programs; researchers can directly compare or even combine different approaches to isolated problems, by asking which agent is best at maximizing a given "goal function".

It also gives them a common language to communicate with other fields—such as mathematical optimization (which is defined in terms of "goals") or economics (which uses the same definition of a "rational agent").[9]


"""

In [9]:
from summarizer import Summarizer

# Create a BERT extractive summarizer
summarizer = Summarizer()

# Generate the summary
summary = summarizer(
    text, min_length=100, max_length=200
)  # You can adjust the min_length and max_length parameters

summary = summary.replace("\n", " ")
summary = summary.strip()
print("Original Text:")
print(text)
print("\nSummary:")
print(summary)

Original Text:
In intelligence and artificial intelligence, an intelligent agent (IA) is an agent that perceives its environment, takes actions autonomously in order to achieve goals, and may improve its performance with learning or acquiring knowledge.  An intelligent agent may be simple or complex: A thermostat or other control system is considered an example of an intelligent agent, as is a human being, as is any system that meets the definition, such as a firm, a state, or a biome.[1]   Simple reflex agent diagram Leading AI textbooks define "artificial intelligence" as the "study and design of intelligent agents", a definition that considers goal-directed behavior to be the essence of intelligence. Goal-directed agents are also described using a term borrowed from economics, "rational agent".[1]  An agent has an "objective function" that encapsulates all the IA's goals. Such an agent is designed to create and execute whatever plan will, upon completion, maximize the expected value

In [7]:
sql = f"""UPDATE tbl_doc_elements SET summary = '{summary}' WHERE id = '{id}';
"""
print(sql)

UPDATE tbl_doc_elements SET summary = 'Further research is necessary to explore the safety, security and societal impacts of AI agents and multi-agent systems, emphasizing both technical solutions and organizational governance frameworks. These efforts are critical for mitigating risks associated with the ongoing development, deployment and increasing use of more sophisticated AI agents in a range of domains.' WHERE id = '1b22edff-bec9-40e8-8c8b-515655dd0377';



In [8]:
cur.execute(sql)
conn.commit()

In [9]:
cur.close()
conn.close()

In [10]:
# cur.close()
# conn.close()